In [2]:
import pandas as pd 
import tensorflow as tf

2024-01-21 18:21:17.723883: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-21 18:21:17.946705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-21 18:21:17.946764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-21 18:21:17.986318: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-21 18:21:18.069435: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-21 18:21:18.070893: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [38]:
df=pd.read_csv("insurance.csv")

In [44]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [53]:
# Separate features and target

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the columns to be one-hot encoded
categorical_cols = ['sex', 'smoker','region']

# Create a column transformer with one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(), categorical_cols)
    ],
    remainder='passthrough'
)

X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)




array([[ 1.   ,  0.   ,  1.   , ..., 46.   , 19.95 ,  2.   ],
       [ 1.   ,  0.   ,  1.   , ..., 47.   , 24.32 ,  0.   ],
       [ 1.   ,  0.   ,  1.   , ..., 52.   , 24.86 ,  0.   ],
       ...,
       [ 0.   ,  1.   ,  1.   , ..., 58.   , 25.175,  0.   ],
       [ 1.   ,  0.   ,  0.   , ..., 37.   , 47.6  ,  2.   ],
       [ 0.   ,  1.   ,  1.   , ..., 55.   , 29.9  ,  0.   ]])

In [63]:
# Model 1
model = Sequential([
   tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='SGD', loss='mae', metrics=['mae'])

# Create a pipeline with encoding and neural network model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

model.fit(X_train_encoded, y_train, epochs=100, batch_size=32, validation_data=(X_test_encoded, y_test),verbose=0)



In [76]:
y_pred = model.predict(X_test_encoded)


9/9 [==============================] - 0s 1ms/step


In [77]:
from sklearn.metrics import mean_absolute_error ,mean_squared_error


mae=mean_absolute_error(y_test, y_pred)
mse= mean_squared_error(y_test, y_pred)
mae,mse

(6999.014962193098, 163170640.1120236)

In [107]:
# Model 2
model_2 = Sequential([
    Dense(64,  input_shape=(X_train_encoded.shape[1],)),
    Dense(32),
    Dense(1 )
])

# Compile the model_2
model_2.compile(optimizer='Adam', loss='mae', metrics=['mae'])

# Create a pipeline with encoding and neural network model_2
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model_2)
])

model_2.fit(X_train_encoded, y_train, epochs=100, batch_size=32, validation_data=(X_test_encoded, y_test),verbose=0)

y_pred_2 = model_2.predict(X_test_encoded)


9/9 [==============================] - 0s 1ms/step


In [108]:
from sklearn.metrics import mean_absolute_error ,mean_squared_error


mae=mean_absolute_error(y_test, y_pred_2)
mse= mean_squared_error(y_test, y_pred_2)
mae,mse


(3763.8291106302104, 59415447.65561795)

In [119]:
# Model 3

# Model 2
model_3 = Sequential([
    Dense(100,  input_shape=(X_train_encoded.shape[1],)),
    Dense(32),
    Dense(1 )
])

# Compile the model_3
model_3.compile(optimizer='Adam', loss='mae', metrics=['mae'])

# Create a pipeline with encoding and neural network model_3
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model_3)
])

model_3.fit(X_train_encoded, y_train, epochs=1000, batch_size=32, validation_data=(X_test_encoded, y_test),verbose=0)

y_pred_3 = model_3.predict(X_test_encoded)


9/9 [==============================] - 0s 2ms/step


In [120]:
from sklearn.metrics import mean_absolute_error ,mean_squared_error


mae=mean_absolute_error(y_test, y_pred_3)
mse= mean_squared_error(y_test, y_pred_3)
mae,mse


(3160.384728541409, 42968165.39272896)

In [128]:
# mode 4


model_4 = tf.keras.Sequential([
    # tf.keras.layers.Dense(300, activation = tf.keras.activations.relu,input_shape=(X_train_encoded.shape[1])),
    tf.keras.layers.Dense(300, activation=tf.keras.activations.relu, input_shape=(X_train_encoded.shape[1],)),

    tf.keras.layers.Dense(200, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(50, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(50, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(10, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(1)
])

# compiling the model_4
model_4.compile(loss = tf.keras.losses.mae, optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01), metrics = ['mae'])

# fit the model_4
model_history = model_4.fit(X_train_encoded, y_train, epochs = 500)

Epoch 1/500
34/34 [==============================] - 1s 3ms/step - loss: 9832.7080 - mae: 9832.7080
Epoch 2/500
34/34 [==============================] - 0s 5ms/step - loss: 7233.7119 - mae: 7233.7119
Epoch 3/500
34/34 [==============================] - 0s 3ms/step - loss: 6535.9238 - mae: 6535.9238
Epoch 4/500
34/34 [==============================] - 0s 4ms/step - loss: 6065.2632 - mae: 6065.2632
Epoch 5/500
34/34 [==============================] - 0s 4ms/step - loss: 4883.0986 - mae: 4883.0986
Epoch 6/500
34/34 [==============================] - 0s 4ms/step - loss: 4316.6865 - mae: 4316.6865
Epoch 7/500
34/34 [==============================] - 0s 3ms/step - loss: 3476.3867 - mae: 3476.3867
Epoch 8/500
34/34 [==============================] - 0s 4ms/step - loss: 3254.1990 - mae: 3254.1990
Epoch 9/500
34/34 [==============================] - 0s 4ms/step - loss: 3215.3174 - mae: 3215.3174
Epoch 10/500
34/34 [==============================] - 0s 4ms/step - loss: 3049.7981 - mae: 3049.7981

In [131]:
model_4.evaluate(X_test_encoded, y_test)


9/9 [==============================] - 0s 2ms/step - loss: 1546.7826 - mae: 1546.7826


[1546.7825927734375, 1546.7825927734375]